# 第一章：快速上手 — 三分钟看到完整分析结果

---

## 物理背景：我们在分析什么？

### 金属-水界面

在很多电化学、催化研究中，我们关心**金属表面与水溶液接触时**发生了什么。  
本项目分析的是：**铜（Cu）银（Ag）合金**金属板，两侧被液态水包围，整个体系在**周期性边界条件**下进行分子动力学（MD）模拟。

示意图（俯视截面，z 轴为垂直方向）：

```
  z=高  ┌────────────────────┐
        │     液态水 (上侧)   │
        │  ←─ 界面 (high_c)  │
        │  Cu Cu Cu Cu Cu Cu │  ← 金属板（4层）
        │  ←─ 界面 (low_c)   │
        │     液态水 (下侧)   │
  z=低  └────────────────────┘
        （周期性：上下连通）
```

### 为什么要分析水的密度和取向？

紧贴金属表面的水分子，其排列方式和指向与体相水截然不同：
- **密度**：界面附近水分子紧密堆积，出现多个密度峰（层状结构）
- **取向**：水分子的 H-O-H 开口方向受金属表面极化影响，产生有序排列

这两种信息对理解电化学双电层、电催化反应机理至关重要。

### 三联图各 panel 回答什么问题？

| Panel | 纵轴 | 横轴 | 物理问题 |
|-------|------|------|----------|
| 上图 | 水的质量密度 (g/cm³) | 距界面距离 (Å) | 水分子在哪里堆积？ |
| 中图 | 取向加权密度 (g/cm³) | 距界面距离 (Å) | 水分子如何定向？ |
| 下图 | 角度概率密度 (a.u.) | 取向角 θ (度) | 吸附层水分子最偏好哪个朝向？ |

---

## 安装说明

运行本教程前，需要先在项目根目录安装本包：

```bash
cd /path/to/MD-analysis
pip install -e .
```

`pip install -e .` 的含义：  
- `-e` = "editable mode"（可编辑模式），修改源码后无需重新安装即可生效  
- `.` = 当前目录（项目根目录）

还需要安装依赖库：
```bash
pip install numpy matplotlib ase
```

## 路径设置

所有 notebook 都从 `tutorial/` 目录运行。  
项目根目录在上一级（`..`），数据文件在 `data_example/potential/` 中。

In [ ]:
import sys
from pathlib import Path

# 项目根目录 = tutorial/ 的上级目录
PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

# 数据目录
DATA_DIR = PROJECT_ROOT / "data_example" / "potential"

# 输出目录（本 notebook 的输出放在 tutorial/output/）
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"项目根目录: {PROJECT_ROOT}")
print(f"数据目录: {DATA_DIR}")
print(f"数据目录中的文件:")
for f in sorted(DATA_DIR.iterdir()):
    print(f"  {f.name}")

## 运行完整分析

只需要**三行代码**，就可以完成从原始轨迹到三联图的全部分析：
1. 导入主函数
2. 指定输入文件路径
3. 调用函数

> ⚠️ **注意**：此步骤需要读取 274 帧轨迹并进行计算，**耗时约 2~5 分钟**，请耐心等待。

In [ ]:
from src.structure.Analysis import plot_water_three_panel_analysis

# 调用主分析函数
png_path = plot_water_three_panel_analysis(
    xyz_path=DATA_DIR / "md-pos-1.xyz",        # 轨迹文件：274 帧 MD 轨迹
    md_inp_path=DATA_DIR / "md.inp",            # 输入文件：晶胞参数
    output_dir=OUTPUT_DIR,                       # 输出目录
)

print(f"分析完成！图片保存在: {png_path}")

## 在 Notebook 中查看结果图

In [ ]:
from IPython.display import Image, display

display(Image(filename=str(png_path), width=700))

## 解读三联图

### Panel 1（上图）：水的质量密度分布

- **横轴**：从金属表面（0 Å）到模拟盒子中央的距离
- **纵轴**：水的质量密度（g/cm³）
- **第一个峰**（约 2~3 Å 处）：**吸附层**（adsorbed layer），紧贴金属表面的水分子，密度远高于体相水
- **体相值**（约 1.0 g/cm³）：远离界面后，密度趋于液态水的正常值
- **物理意义**：金属表面使水分子产生层状有序结构

### Panel 2（中图）：取向加权密度分布

- **纵轴**：$\rho_{H_2O} \cdot \cos\varphi$，其中 $\varphi$ 是水分子 H-O-H 角平分线与 +c 方向的夹角
- **正值**：水分子的 H 端平均朝向 +c（朝上，远离金属）
- **负值**：水分子的 H 端平均朝向 -c（朝下，指向金属）
- **振荡行为**：界面附近水分子有序定向，远处趋于随机（接近零）

### Panel 3（下图）：吸附层水分子取向角概率密度

- **横轴**：取向角 θ（0°到 180°）
- **θ ≈ 0°**：水分子 H 端朝向 +c 方向（H 指向体相水）
- **θ ≈ 180°**：水分子 H 端朝向 -c 方向（H 指向金属）
- **峰的位置**：揭示吸附层水分子最偏好的朝向

---

## 项目整体架构

本项目采用**两层架构**：

```
输入: md-pos-1.xyz (轨迹) + md.inp (晶胞参数)
           │
           ▼
┌─────────────────────────────────────────┐
│  Layer 2: src/structure/Analysis/       │
│  多帧工作流（轨迹读取 + 集成平均）       │
│  ┌─────────────────────────────────┐   │
│  │ Water.py                        │   │
│  │ plot_water_three_panel_analysis │   │
│  └──────────┬──────────────────────┘   │
│             │ 逐帧调用                  │
│  ┌──────────▼──────────────────────┐   │
│  │ Layer 1: src/structure/utils/   │   │
│  │ 单帧低层（LayerParser + WaterParser)│  │
│  └─────────────────────────────────┘   │
└─────────────────────────────────────────┘
           │
           ▼
输出: 5 个 CSV + 1 个 TXT + 1 个 PNG
```

### 输出文件说明

运行完成后，`tutorial/output/` 目录中应该有这些文件：

In [ ]:
import os

print("tutorial/output/ 目录中的输出文件：\n")
for f in sorted(OUTPUT_DIR.iterdir()):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<60s}  ({size_kb:.1f} KB)")

### 各输出文件含义

| 文件名 | 内容 |
|--------|------|
| `water_mass_density_z_distribution_analysis.csv` | 密度剖面数据（距离 + 密度） |
| `water_orientation_weighted_density_z_distribution_analysis.csv` | 取向加权密度剖面 |
| `adsorbed_water_orientation_profile.csv` | 吸附层剖面（含吸附层掩码） |
| `adsorbed_water_theta_distribution_0_180.csv` | 吸附层水分子角度分布 |
| `adsorbed_water_layer_range.txt` | 吸附层边界参数（纯文本） |
| `water_three_panel_analysis.png` | 三联图（主要分析结果） |

---

## 本章小结

✅ 你已经成功运行了完整的 MD-analysis 分析流程！  
✅ 你看到了三联图的物理含义。  
✅ 你了解了项目的两层架构和输出文件。

**下一章**（`02_numpy_matplotlib_ase.ipynb`）将教你理解代码背后使用的三个核心库：NumPy、Matplotlib、ASE。